In [53]:
import pandas as pd
from matplotlib import pyplot as plt
import os
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from itertools import product
import time
from itertools import product, combinations
from sklearn.decomposition import NMF

In [55]:
data_dir = os.path.dirname(os.getcwd()) + '/data/'
files = os.listdir(data_dir)

train = pd.read_csv(data_dir + 'sales_train.csv')
test = pd.read_csv(data_dir + 'test.csv')

shops = pd.read_csv(data_dir + 'shops.csv')
item_cat = pd.read_csv(data_dir + 'item_categories.csv')
item = pd.read_csv(data_dir + 'items.csv')
sample_sub = pd.read_csv(data_dir + 'sample_submission.csv')

### Data cleaning

In [60]:
# from https://www.kaggle.com/dlarionov/feature-engineering-xgboost

##### fix shop ids (duplicate shops)#####

# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

##### remove price and daily connt outliers ####

train = train[(train.item_price<100000) & (train.item_price>0)]
train = train[(train.item_cnt_day<1001) & (train.item_cnt_day>=0)]

#### fill pirce below zero with meadian ####

median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
train.loc[train.item_price<0, 'item_price'] = median

### select those shop and items that appears in test. (think of ways to utilize those unpresanted later)

In [10]:
# train_lk = train[train.shop_id.isin(test.shop_id.unique())]
# train_lk = train_lk[train_lk.item_id.isin(test.item_id.unique())]
# print('train_lk shape: ', train_lk.shape)

train_lk shape:  (1227789, 6)


### sum cnts over the whole time to prepare the recommender system like matrix

In [61]:
train_sum = train.groupby(['shop_id', 'item_id'], as_index=False).agg({'item_cnt_day': 'sum'})
train_sum = train_sum.rename({'item_cnt_day': 'item_cnt_sum'}, axis=1)
train_sum.shape

(418882, 3)

In [62]:
train_pivot = train_sum.pivot_table(index='shop_id', columns='item_id', values='item_cnt_sum', aggfunc=np.sum,
                                   fill_value=0)
train_pivot.head()

item_id,0,1,2,3,4,5,6,7,8,9,...,22160,22161,22162,22163,22164,22165,22166,22167,22168,22169
shop_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,18,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,7,0,0,5,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,4,0,3,0,0,4,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,12,2,10,0,0,4,0,0
6,0,0,0,0,0,0,0,0,0,0,...,3,0,10,1,14,0,0,24,0,0


In [63]:
n_components = 5

model = NMF(n_components=n_components, init='random', random_state=0)
W = model.fit_transform(train_pivot)
H = model.components_

In [64]:
shop_feature = pd.DataFrame(W, index=train_pivot.index).reset_index()
shop_feature.columns = ['shop_id'] + ['shop_latent_feature_' + str(i) for i in range(n_components)]

item_feature = pd.DataFrame(H.transpose(), index=train_pivot.columns).reset_index()
item_feature.columns = ['item_id'] + ['item_latent_feature_' + str(i) for i in range(n_components)]

### Save latent feature to pickle file

In [65]:
shop_feature.to_pickle(data_dir+'shop_feature_'+str(n_components)+'.pkl')
item_feature.to_pickle(data_dir+'item_feature_'+str(n_components)+'.pkl')